In [ ]:
#Import packages
import sys
sys.path.insert(0, '') ## Insert in the quotes the path to where QuantSIM folder is in your computer.

import numpy as np
import pandas as pd
import re
import os
import copy
import matplotlib.pyplot as plt
from pathlib import Path
import smbclient
import time
import shutil
import itertools
import bisect
from czifile import imread, CziFile
import napari
import scipy.ndimage as scipy_image
import pyclesperanto_prototype as cle
import scipy
import seaborn as sns
import xml.etree.ElementTree as ET
import warnings
from tqdm import tqdm
import traceback
import gzip

from QuantSIM.common import check_to_make_dir
from QuantSIM.SIM_segment import SIM_segmentation, slide_channel_threshold
from QuantSIM.SIM_analyze import SIM_analysis

In [ ]:
##SIM_BLL_syncHU

# Identification of nuclei by automated segmentation of DAPI single slices, separate files from main 3D-SIM stacks.
# 2 channels: EdU (clicked, AF647), Cy3-dUTP.

## Insert in the quotes the path to where you store 3D-SIM stack files (czi format) are in your computer.
# The expected paths are:
# 3D-SIM stack files: MAIN_PATH/RAWS_FOLDER/EXP_FOLDER/slide/ _Out.czi files
# 3D-SIM DAPI files: MAIN_PATH/DAPI_FOLDER/EXP_FOLDER/slide/ _dapi.czi files
# output files: MAIN_PATH/RES/FOLDER/EXP_FOLDER_AUTO_ANALYSIS/slide/ many files organized in layers (viewable with Napari), tables, overlap_layers, overlap_tables

MAIN_PATH = ''
RAWS_FOLDER = 'raw_files'
DAPI_FOLDER = 'raw_files'
EXP_FOLDER = 'SIM_BLL_syncHU'
RES_FOLDER = 'segmented/OTSU_THRESHOLDING'

EXP_RAWS_PATH = os.path.join(MAIN_PATH, RAWS_FOLDER, EXP_FOLDER)
EXP_DAPI_PATH = os.path.join(MAIN_PATH, DAPI_FOLDER, EXP_FOLDER)
RES_PATH = os.path.join(MAIN_PATH, RES_FOLDER, EXP_FOLDER + '_AUTO_ANALYSIS')
#RES_MAIN_FOLDER = EXP_PATH[::-1].split('/', 1)[0][::-1] + '_AUTO_ANALYSIS' ## Double reverse search: Reverse to pick last element after splitting (which is first) and reverse again to get original substring.
cle.select_device('M1')
if not os.path.exists(EXP_RAWS_PATH):
    raise Exception('The experimental data path indicated could not be found.')

if not os.path.exists(EXP_DAPI_PATH):
    raise Exception('Nuclei masks path indicated could not be found.')

if not os.path.exists(RES_PATH):
    check_to_make_dir(RES_PATH, known_path = MAIN_PATH)

EXISTING_PATH = RES_PATH
threshold_method = 'otsu'
spot_sigma = 0
outline_sigma = 0
min_size = 50 # Considering a pixel as a cube, volume approx. 16000 nm**3. Nucleosome volume is 665 nm**3, 24 fully-packed nucleosomes/pixel volume (but chromatin status, signal intensity and light diffraction may alter nucleosome relative size).
seg_keywords = ['raw','trim']
min_size_overlap = 0
pixel_to_nano_conversion = 25
unit = 'nano'

foldernames = ['raw','trim']

for foldername,seg_keyword in zip(foldernames,seg_keywords):
    RES_FOLDERNAME =  foldername+'__ss_{}__os_{}__min_size_{}__min_size_overlap_{}'.format(str(spot_sigma),str(outline_sigma), str(min_size), str(min_size_overlap))
    RES_FOLDERNAME = os.path.join(RES_PATH, RES_FOLDERNAME)
    for slide in os.listdir(EXP_RAWS_PATH):
        SLIDE_PATH = os.path.join(EXP_RAWS_PATH, slide)
        SLIDE_DAPI = os.path.join(EXP_DAPI_PATH, slide)
        if os.path.isdir(SLIDE_PATH) and os.path.isdir(SLIDE_DAPI):
            czi_files = np.array(os.listdir(SLIDE_PATH))
            czi_files = np.array(czi_files[['_Out.czi' in czi_file for czi_file in czi_files]])
            dapi_files = np.array(os.listdir(SLIDE_DAPI))
            dapi_files = np.array(dapi_files[['_dapi.czi' in dapi_file for dapi_file in dapi_files]])
            if len(czi_files) == 0:
                continue
            ## make sure they are in order
            def extract_file_number(filename, regex):
                return int(re.match(regex,filename).group(1))
            czi_files = sorted(czi_files, key = lambda filename: extract_file_number(filename, regex = r'Image (\d+).*\.czi$'))
            dapi_files = sorted(dapi_files, key = lambda filename: extract_file_number(filename, regex = r'Image (\d+).*\.czi$'))
            # Link every image with every mask that share ID
            linked_files_slide = np.array(list(zip(czi_files,dapi_files)))   ## Nested list. Lists inside have two elements: processed czi filename, dapi filename
            for linked_files_image in linked_files_slide:
                czi_file = linked_files_image[0]
                dapi_file = linked_files_image[1]
                RES_IMAGE_PATH = os.path.join(RES_FOLDERNAME, slide, czi_file[:-4]) # -4 because we remove .czi for naming the folder. Added mask_match to not overwrite images with multiple masks.
                if os.path.isdir(RES_IMAGE_PATH):
                    continue
                check_to_make_dir(RES_IMAGE_PATH)
                EXISTING_PATH = RES_IMAGE_PATH
                DAPI_PATH = os.path.join(SLIDE_DAPI, dapi_file)
                IMAGE_PATH = os.path.join(SLIDE_PATH, czi_file)
                channel_names = ['647_dye','555_dye']#,'488_dye']
                channel_colors = ['magenta','yellow']#,'green']
                dist_calculation_1 = dict(zip(channel_names,['centroid','centroid','centroid']))
                dist_calculation_2 = dict(zip(channel_names,['maxima','maxima','maxima']))
                channel_laser_wavelengths = []
                metadata = CziFile(IMAGE_PATH)
                xml_metadata = ET.ElementTree(ET.fromstring(metadata.metadata()))
                root = xml_metadata.getroot()
                for element in xml_metadata.iter('ExcitationWavelength'):
                    channel_laser_wavelengths.append(element.text)
                channel_laser_wavelengths = channel_laser_wavelengths[:int(round(len(channel_laser_wavelengths)/2,0))]
                channel_laser_wavelengths = [int(round(float(i),0)) for i in channel_laser_wavelengths]
                wavelengths_np = np.array(channel_laser_wavelengths)
                channel_org = list(range(len(channel_laser_wavelengths)))
                channel_org = ['skip_channel' for i in channel_org]
                channel_org[np.where(wavelengths_np == 642)[0][0]] = '647_dye'
                channel_org[np.where(wavelengths_np == 561)[0][0]] = '555_dye'
                #channel_org[np.where(wavelengths_np == 488)[0][0]] = '488_dye'
                Exp = SIM_segmentation(IMAGE_PATH, RES_IMAGE_PATH,
                                    channel_names = channel_names,
                                    channel_colors = channel_colors,
                                    channel_org = channel_org,
                                    DAPI_PATH = DAPI_PATH,
                                    override = True)
                Exp.anisotropy_correction()
                Exp.image_truncation()
                Exp.make_spatial_mask(prism_mode = True)
                ## We are reusing the global threshold from whole image to do segmentation. The other option is redo the threshold in the cropped image, which may have slight changes (or a lot if the cell was dim in a channel).
                Exp.segmentation_channel_intensity_based('647_dye', spot_sigma = spot_sigma, outline_sigma = outline_sigma,
                                                         fit_threshold_to_signal_mask=1000, use_roi_mask = True) #inherit_threshold = Exp.channels_threshold_trunc['647_dye'],
                Exp.segmentation_channel_intensity_based('555_dye', spot_sigma = spot_sigma, outline_sigma = outline_sigma,
                                                         fit_threshold_to_signal_mask=1000, use_roi_mask = True) #inherit_threshold = Exp.channels_threshold_trunc['555_dye'],
                #Exp.segmentation_channel_intensity_based('488_dye', spot_sigma = spot_sigma, outline_sigma = outline_sigma,
                #                                         fit_threshold_to_signal_mask=1000, use_roi_mask = True) #inherit_threshold = Exp.channels_threshold_trunc['488_dye'],
                Exp.select_segmentation(seg_keyword)
                Exp.save_segmentation_layers(prefix='ALL_REGIONS__')
                Exp.remove_small_regions('647_dye',seg_keyword, min_size)
                Exp.remove_small_regions('555_dye',seg_keyword, min_size)
                #Exp.remove_small_regions('488_dye',seg_keyword, min_size)
                ## Check if Exp has regions. If empty, skip to next image.
                Exp.select_segmentation(seg_keyword)
                if any(len(Exp.tables['004_segmented_regions'][channel].index) < 2 for channel in channel_names):
                    continue
                Exp.dist_to_core_and_rim()
                Exp.save_segmentation_layers()
                with open(os.path.join(RES_IMAGE_PATH,'000 SIM_segmentation extra info.txt'),'w') as txtfile:
                    txtfile.write(str(dict(zip(channel_names,list(zip(channel_colors,channel_laser_wavelengths))))))
                    txtfile.write('\nScaling: {}'.format(str(Exp.voxelsizes)))
                    txtfile.write('\nInitial threshold method to find overall signal regions: {}'.format(threshold_method))
                    txtfile.write('\nThreshold calculated from max-z projection for truncation:')
                    txtfile.write('\n'+str(Exp.channels_threshold_trunc))
                    txtfile.write('\nCoordinates of truncation corner reference (YX):')
                    txtfile.write('\n'+str(Exp.trunc_refcoord))
                    txtfile.write('\nThreshold calculated from max-z projection for segmentation:')
                    txtfile.write('\n'+str(Exp.channels_threshold_seg))
                    txtfile.write('\nSpot sigma (local maxima identification)')
                    txtfile.write('\n'+str(spot_sigma))
                    txtfile.write('\nOutline sigma (Voronoi-watershed)')
                    txtfile.write('\n'+str(outline_sigma))
                    txtfile.write('\nMinimum segmented region size (pixels)')
                    txtfile.write('\n'+str(min_size))
                    txtfile.write('\nSegmentation selected')
                    txtfile.write('\n'+str(seg_keyword))
                    txtfile.write('\nMinimum overlap size (pixels)')
                    txtfile.write('\n'+str(min_size_overlap))
                    txtfile.write('\nunit conversion factor: ')
                    txtfile.write('\n'+str(pixel_to_nano_conversion))
                    txtfile.write('\nunit converted to: ')
                    txtfile.write('\n'+str(unit))
                for channel in channel_names:
                    Exp_channel = SIM_analysis(Exp, channel_1 = channel, channel_2 = channel)
                    Exp_channel.neighbor_distance()
                    Exp_channel.neighbor_distance(channels_ref = ['maxima','maxima'])
                    Exp_channel.overlap_detection()
                    Exp_channel.overlap_declumping()
                    Exp_channel.two_channel_overlap_regions(overlap_stoich=(1,1))
                    Exp_channel.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    Exp_channel.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_channel.single_channel_cleanup()
                    Exp_channel.save_segmentation_layers(prefix = 'one_channel__')
                    del Exp_channel
                for combo in itertools.combinations(channel_names, 2):
                    Exp_overlap = SIM_analysis(Exp, channel_1 = combo[0], channel_2 = combo[1])
                    Exp_overlap.neighbor_distance(channels_ref = [dist_calculation_1[combo[0]],dist_calculation_1[combo[1]]])
                    Exp_overlap.neighbor_distance(channels_ref = [dist_calculation_2[combo[0]],dist_calculation_2[combo[1]]])
                    Exp_overlap.overlap_detection(min_size_overlap = min_size_overlap)
                    if all(['EdU' in combo,'dUTP' in combo]):
                        Exp_overlap.overlap_declumping()
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_common__')
                    ## PROBLEM: keep calculating things when they have no data, no overlaps to work with, should be empty but then gets confused...
                    ## SOLUTION: ignore instances where there are less than 2 intersecting regions identified.
                    if len(Exp_overlap.tables['102_intersection_regions'].index) < 2:
                        with open(os.path.join(RES_IMAGE_PATH,'000 SIM_analysis {}_vs_{} extra info.txt'.format(str(combo[0]),str(combo[1]))), 'w') as txtfile:
                            txtfile.write('Channels evaluated for overlap: {}'.format(str(combo)))
                            for i in range(len(combo)):
                                txtfile.write('\n'+str(combo[i]))
                                txtfile.write('\n\tValue\tCount')
                                txtfile.write('\n'+str(Exp_overlap.tables['102_overlap_bool'].sum(axis = 1-i).value_counts().sort_index()))
                                txtfile.write('\nTotal: '+ str(Exp.tables['004_segmented_maxima'][combo[i]].shape[0]))
                            txtfile.write('\nFork structure number')
                            txtfile.write('\nInitiation: NA')
                            txtfile.write('\nOngoing: NA')
                            txtfile.write('\nTermination: NA')
                        del Exp_overlap
                        continue
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(0,0))
                    Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    #Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_no_overlap__', common_results = False)
                    del Exp_overlap.layers['105_stoich_0-to-0_segmented_labeled']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(1,1))
                    Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = 1, unit = 'pixel')
                    #Exp_overlap.measure_overlapping_regions(pixel_unit_conversion = pixel_to_nano_conversion, unit = unit)
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_ongoing__', common_results = False)
                    ong_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    del Exp_overlap.layers['105_stoich_1-to-1_segmented_labeled']
                    del Exp_overlap.tables['106_channel_measurements']
                    del Exp_overlap.tables['106_overlap_measurements']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(2,1))
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_initiation__', common_results = False)
                    init_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    del Exp_overlap.layers['105_stoich_2-to-1_segmented_labeled']
                    Exp_overlap.two_channel_overlap_regions(overlap_stoich=(1,2))
                    Exp_overlap.save_segmentation_layers(prefix = 'two_channel_termination__', common_results = False)
                    term_n = Exp_overlap.tables['105_overlap_labels'].shape[0]
                    with open(os.path.join(RES_IMAGE_PATH,'000 SIM_analysis {}_vs_{} extra info.txt'.format(str(combo[0]),str(combo[1]))), 'w') as txtfile:
                        txtfile.write('Channels evaluated for overlap: {}'.format(str(combo)))
                        for i in range(len(combo)):
                            txtfile.write('\n'+str(combo[i]))
                            txtfile.write('\n\tValue\tCount')
                            txtfile.write('\n'+str(Exp_overlap.tables['102_overlap_bool'].sum(axis = 1-i).value_counts().sort_index()))
                            txtfile.write('\nTotal: '+ str(Exp.tables['004_segmented_maxima'][combo[i]].shape[0]))
                        txtfile.write('\nFork structure number')
                        txtfile.write('\nInitiation: '+ str(init_n))
                        txtfile.write('\nOngoing: '+ str(ong_n))
                        txtfile.write('\nTermination: '+ str(term_n))
                    del Exp_overlap
                del Exp

Path created: /Users/brunourien/SciLifeLab/LemmensLab/Image Analysis/SIM_Analysis/segmented/OTSU_THRESHOLDING/250603_SIM_BLL_BU_AUTO_ANALYSIS/raw__ss_0__os_0__min_size_50__min_size_overlap_0/slide 3/Image 38_Out
--- Output folder: ../../../../../LemmensLab/Image Analysis/SIM_Analysis/segmented/OTSU_THRESHOLDING/250603_SIM_BLL_BU_AUTO_ANALYSIS/raw__ss_0__os_0__min_size_50__min_size_overlap_0/slide 3/Image 38_Out
Seconds elapsed, anisotropy correction: 23.48
Seconds elapsed, image truncation: 11.8785


/opt/anaconda3/envs/QuantSIM/lib/python3.10/site-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int32 to uint8 without scaling because max value 1 fits in uint8
  return _convert(image, np.uint8, force_copy)


Seconds elapsed, making spatial mask: 16.4252
INITIAL THRESHOLD: 0.1410543
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(955, 898) 529337 256281 15 71957
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 33083.625
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(955, 898) 529337 256281 15 71957
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 33083.625
FINAL THRESHOLD: 0.1410543
size: 50
size: 50
size: 50
size: 50
size: 50
size: 50
Seconds elapsed, intensity-based segmentation of 647_dye: 134.0004
INITIAL THRESHOLD: 0.100971125
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(955, 898) 529333 262396 19 65842
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 26466.7
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(955, 898) 529333 262396 19 65842
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 26466.7
FINAL THRESHOLD: 0.100971125
size: 5

/opt/anaconda3/envs/QuantSIM/lib/python3.10/site-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int32 to uint8 without scaling because max value 1 fits in uint8
  return _convert(image, np.uint8, force_copy)


Seconds elapsed, making spatial mask: 14.4767
INITIAL THRESHOLD: 0.106655344
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1085, 728) 444205 245446 2814 97415
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 157.79964476021314
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1085, 728) 446575 315417 444 27444
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 1003.5415730337079
FINAL THRESHOLD: 0.2222553443670171
size: 46
size: 46
size: 46
size: 46
size: 46
size: 46
Seconds elapsed, intensity-based segmentation of 647_dye: 53.6958
INITIAL THRESHOLD: 0.04896552
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1085, 728) 444430 270778 2589 72083
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 171.5949806949807
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1085, 728) 446574 327942 445 14919
BACKGROUND, NO SIGNAL TO SIGNAL RATI

...

Seconds elapsed, making spatial mask: 17.4317
INITIAL THRESHOLD: 0.01703784
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1076, 798) 461181 303585 1908 91974
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 241.58302776322682
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1076, 798) 462631 327095 458 68464
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 1007.9128540305011
FINAL THRESHOLD: 0.021337840560078595
size: 46
size: 46
size: 46
size: 46
size: 46
size: 46
Seconds elapsed, intensity-based segmentation of 647_dye: 67.1633
INITIAL THRESHOLD: 0.025545754
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1076, 798) 462942 299944 147 95615
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 3127.9932432432433
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1076, 798) 462942 299944 147 95615
BACKGROUND, NO SIGNAL TO SIGNAL RA

/opt/anaconda3/envs/QuantSIM/lib/python3.10/site-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 1 fits in uint8
  return _convert(image, np.uint8, force_copy)


Seconds elapsed, making spatial mask: 16.9456
INITIAL THRESHOLD: 0.013653025
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1100, 725) 434931 264893 11412 86264
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 38.10847279418207
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1100, 725) 445899 320360 444 30797
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 1002.0224719101124
FINAL THRESHOLD: 0.025253025031089713
size: 48
size: 48
size: 48
size: 48
size: 48
size: 48
Seconds elapsed, intensity-based segmentation of 647_dye: 61.5831
INITIAL THRESHOLD: 0.019276597
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1100, 725) 446343 272496 0 78661
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 446344.0
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(1100, 725) 446343 272496 0 78661
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 446344.0

/opt/anaconda3/envs/QuantSIM/lib/python3.10/site-packages/skimage/util/dtype.py:541: UserWarning: Downcasting int64 to uint8 without scaling because max value 1 fits in uint8
  return _convert(image, np.uint8, force_copy)


Seconds elapsed, making spatial mask: 13.4245
INITIAL THRESHOLD: 0.026707254
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(731, 939) 382786 260465 138 43020
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 2753.863309352518
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(731, 939) 382786 260465 138 43020
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 2753.863309352518
FINAL THRESHOLD: 0.026707254
size: 46
size: 46
size: 46
size: 46
size: 46
size: 46
Seconds elapsed, intensity-based segmentation of 647_dye: 77.6217
INITIAL THRESHOLD: 0.050908692
BEFORE ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(731, 939) 382860 265332 64 38153
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 5890.169230769231
AFTER ADJUSTING (background - no signal; mask - no signal; background - signal; mask - signal)
(731, 939) 382860 265332 64 38153
BACKGROUND, NO SIGNAL TO SIGNAL RATIO: 5890.16923076